<a href="https://colab.research.google.com/github/darkking019/if03bd2/blob/main/trabalhoBDP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
from sqlite3 import Error

## Função de alteração (insert, update, delete...)
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print(f"Query {query.split('(')[0].strip()} executada.")
        if cursor.rowcount != -1: print(f"{cursor.rowcount} linha(s) afetadas")

    except Error as e:
        print(f"Erro: '{e}'")


# Função de leitura de dados (select)
def execute_read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()

        return result
    except Error as e:
        print(f"Erro: '{e}'")

In [ ]:
## Criando conexão
connection = sqlite3.connect("mapeamento_cultural.sqlite")

query = """
CREATE TABLE endereco (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    rua VARCHAR(100),
    numero VARCHAR(10),
    CEP VARCHAR(10),
    regiao VARCHAR(50)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE pontos_historicos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    descricao TEXT,
    endereco_id INT,
    FOREIGN KEY (endereco_id) REFERENCES endereco(id)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE grupo_cultural (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome VARCHAR(100),
    descricao TEXT,
    fotos TEXT,
    contato VARCHAR(100)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE evento (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    data DATE,
    horario TIME,
    local VARCHAR(100),
    status VARCHAR(50),
    url_ingresso VARCHAR(255)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE localiza (
    grupo_id INTEGER,
    ponto_id INTEGER,
    PRIMARY KEY (grupo_id, ponto_id),
    FOREIGN KEY (grupo_id) REFERENCES grupo_cultural(id),
    FOREIGN KEY (ponto_id) REFERENCES pontos_historicos(id)
);
"""
execute_query(connection, query)

query = """
CREATE TABLE apresenta (
    grupo_id INTEGER,
    evento_id INTEGER,
    grupo_responsavel VARCHAR(100),
    PRIMARY KEY (grupo_id, evento_id),
    FOREIGN KEY (grupo_id) REFERENCES grupo_cultural(id),
    FOREIGN KEY (evento_id) REFERENCES evento(id)
);
"""
execute_query(connection, query)

Query CREATE TABLE endereco executada.
Query CREATE TABLE pontos_historicos executada.
Query CREATE TABLE grupo_cultural executada.
Query CREATE TABLE evento executada.
Query CREATE TABLE localiza executada.
Query CREATE TABLE apresenta executada.


 atividade 1 :Objetivo da consulta : Contar o número de eventos por regiãono site do Projeto Cultural Neves.
 ação esperada : Retorna o número total de eventos por região, usando JOIN com endereco e pontos_historicos.

In [ ]:
query1 = """
SELECT e.regiao, COUNT(ev.id) AS num_eventos
FROM endereco e
JOIN pontos_historicos ph ON e.id = ph.endereco_id
JOIN localiza l ON ph.id = l.ponto_id
JOIN apresenta a ON l.grupo_id = a.grupo_id
JOIN evento ev ON a.evento_id = ev.id
GROUP BY e.regiao;
"""
result1 = execute_read_query(connection, query1)
print("Consulta 1 - Eventos por região:", result1)

Consulta 1 - Eventos por região: []


 atividade 2:
Objetivo da consulta: Calcular a média de eventos por grupo cultural no site.
 ação esperada : Retorna a média de eventos associados a cada grupo cultural.

In [ ]:
query2 = """
SELECT AVG(num_eventos) AS media_eventos_por_grupo
FROM (
    SELECT g.id, COUNT(a.evento_id) AS num_eventos
    FROM grupo_cultural g
    LEFT JOIN apresenta a ON g.id = a.grupo_id
    GROUP BY g.id
) subquery;
"""
result2 = execute_read_query(connection, query2)
print("Consulta 2 - Média de eventos por grupo:", result2)

atividade 3:
 Objetivo da consulta: Listar eventos, grupos culturais e endereços dos pontos históricos associados, no site do Projeto Cultural Neves.
 ação esperada : Retorna detalhes de eventos, incluindo o nome do grupo cultural, o local do evento e o endereço do ponto histórico associado.

In [ ]:
query3 = """
SELECT ev.data, ev.local, g.nome AS grupo, e.rua, e.numero, e.regiao
FROM evento ev
JOIN apresenta a ON ev.id = a.evento_id
JOIN grupo_cultural g ON a.grupo_id = g.id
JOIN localiza l ON g.id = l.grupo_id
JOIN pontos_historicos ph ON l.ponto_id = ph.id
JOIN endereco e ON ph.endereco_id = e.id;
"""
result3 = execute_read_query(connection, query3)
print("Consulta 3 - Eventos, grupos e endereços:", result3)

Consulta 3 - Eventos, grupos e endereços: []


Atividade 04
 consulta  View : A view é necessária para facilitar a consulta recorrente de eventos ativos com detalhes de grupos culturais e locais, otimizando a exibição de eventos confirmados no site.
ação esperada ao rodar a consulta: Retorna eventos com status 'Confirmado', incluindo o nome do grupo cultural e o local.

In [ ]:
create_view = """
CREATE VIEW IF NOT EXISTS eventos_confirmados AS
SELECT ev.id, ev.data, ev.horario, ev.local, g.nome AS grupo_responsavel
FROM evento ev
JOIN apresenta a ON ev.id = a.evento_id
JOIN grupo_cultural g ON a.grupo_id = g.id
WHERE ev.status = 'Confirmado';
"""
execute_query(connection, create_view)

query_view = """
SELECT * FROM eventos_confirmados;
"""
result_view = execute_read_query(connection, query_view)
print("View - Eventos confirmados:", result_view)

Query CREATE VIEW IF NOT EXISTS eventos_confirmados AS
SELECT ev.id, ev.data, ev.horario, ev.local, g.nome AS grupo_responsavel
FROM evento ev
JOIN apresenta a ON ev.id = a.evento_id
JOIN grupo_cultural g ON a.grupo_id = g.id
WHERE ev.status = 'Confirmado'; executada.
View - Eventos confirmados: []


Atividade 04
operação  Procedure : Registrar a associação de um grupo cultural a um evento é uma operação comum no site, e encapsular a lógica em uma procedure garante consistência na inserção e validação de dados.
ação esperada : Insere um registro na tabela apresenta, associando um grupo a um evento, após verificar a existência de ambos.

In [ ]:
def associar_grupo_evento(connection, grupo_id, evento_id, grupo_responsavel):
    check_grupo = f"SELECT id FROM grupo_cultural WHERE id = {grupo_id};"
    check_evento = f"SELECT id FROM evento WHERE id = {evento_id};"
    grupo = execute_read_query(connection, check_grupo)
    evento = execute_read_query(connection, check_evento)
    if grupo and evento:
        insert_query = f"""
        INSERT INTO apresenta (grupo_id, evento_id, grupo_responsavel)
        VALUES ({grupo_id}, {evento_id}, '{grupo_responsavel}');
        """
        execute_query(connection, insert_query)
    else:
        print("Grupo ou evento não encontrado.")

# Exemplo de execução da procedure
associar_grupo_evento(connection, 1, 2, 'Grupo Folclórico Raízes')

Grupo ou evento não encontrado.


 Atividade 05
operação s Function: Verificar o status de um evento (ativo ou pendente) é útil para várias consultas no site, e encapsular em uma function permite reutilização em diferentes contextos.
acao esperada: Retorna 1 se o evento está confirmado, 0 caso contrário.

In [ ]:
def verificar_status_evento(status):
    return 1 if status == 'Confirmado' else 0

connection.create_function("verificar_status_evento", 1, verificar_status_evento)

query_function = """
SELECT id, data, local, verificar_status_evento(status) AS is_confirmado
FROM evento;
"""
result_function = execute_read_query(connection, query_function)
print("Function - Status de eventos:", result_function)

Function - Status de eventos: []


Atividade 06
operação Trigger : Garantir que o status de um evento seja atualizado automaticamente para 'Confirmado' quando associado a um grupo cultural melhora a eficiência e consistência no site.
 disparador do Trigger e das operações que ele executa: Dispara após INSERT em apresenta, atualizando o status do evento correspondente para 'Confirmado'.

In [ ]:
create_trigger = """
CREATE TRIGGER IF NOT EXISTS atualiza_status_evento
AFTER INSERT ON apresenta
FOR EACH ROW
BEGIN
    UPDATE evento
    SET status = 'Confirmado'
    WHERE id = NEW.evento_id;
END;
"""
execute_query(connection, create_trigger)

# Exemplo de teste para acionar o trigger
associar_grupo_evento(connection, 2, 1, 'Banda Cultural Neves')

Query CREATE TRIGGER IF NOT EXISTS atualiza_status_evento
AFTER INSERT ON apresenta
FOR EACH ROW
BEGIN
    UPDATE evento
    SET status = 'Confirmado'
    WHERE id = NEW.evento_id;
END; executada.
Grupo ou evento não encontrado.
